In [1]:
from langchain_fireworks import ChatFireworks
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph, StateGraph
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import List, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages.base import BaseMessage
from typing import TypedDict, Annotated
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatFireworks(model="accounts/fireworks/models/firefunction-v1")

In [2]:
graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [3]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='dbf71680-9847-4a69-ab1d-7cd27316f4af'),
 AIMessage(content='The answer is 2.', response_metadata={'token_usage': {'prompt_tokens': 67, 'total_tokens': 75, 'completion_tokens': 8}, 'model_name': 'accounts/fireworks/models/firefunction-v1', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e9afa14-d2ae-4a09-bffe-cd57f78216cc-0')]

# Interaction with LCEL

As an aside for those already familiar with LangChain - `add_node` actually takes any function or [runnable](https://python.langchain.com/docs/expression_language/interface/) as input. In the above example, the model is used "as-is", but we could also have passed in a function:

In [16]:
def call_oracle(messages: list):
    return model.invoke(messages)

graph.add_node("oracle", call_oracle)

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Node `oracle` already present.

Just make sure you are mindful of the fact that the input to the [runnable](https://python.langchain.com/docs/expression_language/interface/) is the **entire current state**. So this will fail:

In [17]:
# This will not work with MessageGraph!
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant named {name} who always speaks in pirate dialect"),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model

# State is a list of messages, but our chain expects a dict input:
#
# { "name": some_string, "messages": [] }
#
# Therefore, the graph will throw an exception when it executes here.
graph.add_node("oracle", chain)

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Node `oracle` already present.

# Conditional edges

In [18]:
@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

model_with_tools = model.bind_tools([multiply])

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])

builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

builder.add_conditional_edges("oracle", router)

runnable = builder.compile()

In [19]:
runnable.invoke(HumanMessage("What is 123 * 456?"))

[HumanMessage(content='What is 123 * 456?', id='004ca911-bf64-4410-8e5a-c3552a0318f9'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_WQTTJx4O1P7jUd0vCIrfNlIH', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"first_number": 123, "second_number": 456}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 211, 'total_tokens': 243, 'completion_tokens': 32}, 'model_name': 'accounts/fireworks/models/firefunction-v1', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dea24fd6-518a-4827-9176-f5a26e3876a1-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_WQTTJx4O1P7jUd0vCIrfNlIH'}]),
 ToolMessage(content='56088', name='multiply', id='9945b21a-00bf-40f8-bf58-b0e777010a62', tool_call_id='call_WQTTJx4O1P7jUd0vCIrfNlIH')]

In [10]:
runnable.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', id='86db328c-d238-4735-a707-32ce61cb432f'),
 AIMessage(content="I am an AI and I don't have a personal name. I am here to assist you with any questions or tasks you have.", response_metadata={'token_usage': {'prompt_tokens': 204, 'total_tokens': 234, 'completion_tokens': 30}, 'model_name': 'accounts/fireworks/models/firefunction-v1', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea97337f-44da-4a1a-9116-6a14704456c0-0')]

# Cycles

In [3]:
# !pip install -U langgraph langchain_openai tavily-python
# !pip install langchain_community

In [20]:
tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)
model = ChatFireworks(model="accounts/fireworks/models/firefunction-v1", temperature=0)
model = model.bind_tools(tools)

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

# Define the function that determines whether to continue or not
def should_continue(state: AgentState) -> Literal["action", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "action" node
    if last_message.tool_calls:
        return "action"
    # Otherwise, we stop (reply to the user)
    return "__end__"

# Define the function that calls the model
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [21]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jYMnNrIa0oZ7bARyAxWNb9FJ', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 203, 'total_tokens': 232, 'completion_tokens': 29}, 'model_name': 'accounts/fireworks/models/firefunction-v1', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41528e29-30bf-47cf-b6d2-e371a30fb98f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_jYMnNrIa0oZ7bARyAxWNb9FJ'}]),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 17156904

# Streaming

In [22]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs, stream_mode="updates"):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_DFXYpsAVYZXwLne4cIQnccaD', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 203, 'total_tokens': 232, 'completion_tokens': 29}, 'model_name': 'accounts/fireworks/models/firefunction-v1', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4135a8fd-99b2-4f88-be7e-90bae80acece-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_DFXYpsAVYZXwLne4cIQnccaD'}])]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'

In [23]:
model = ChatFireworks(model="accounts/fireworks/models/firefunction-v1")

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
async for output in app.astream_log(inputs, include_types=["llm"]):
    # astream_log() yields the requested logs (here LLMs) in JSONPatch format
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            print(op["value"])

content='' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content='I' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content="'m sorry" id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=',' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' but I don' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content="'t" id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' have the ability' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' to provide weather' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' information at the' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' moment. Please' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' try' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' again' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content=' later.' id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
content='' response_metadata={'finish_reason': 'stop'} id='run-28fad127-971c-4dc2-86a3-1ebf2499e535'
